In [1]:
import os

import gradio as gr
import openai
from langchain.vectorstores import Chroma
from langchain.embeddings.azure_openai import AzureOpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferWindowMemory

from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain


# The LLM
llm = AzureChatOpenAI(
    streaming=True,
    callbacks=CallbackManager([StreamingStdOutCallbackHandler()]),
    temperature=0.0,
    deployment_name='gpt-4o',
)

# The Prompt
template = """

    ...

"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


# The Memory
buffer_window_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", return_messages=True, k=4
)

# The Embeddings
openai_embeddings = AzureOpenAIEmbeddings(
    deployment='text-embedding-3-large'    
)

# The Knowledge Base
persist_directory = "vector_database/chromadb/"
vectordb = Chroma(
    persist_directory=persist_directory, embedding_function=openai_embeddings
)

# The Retriever
retriever = vectordb.as_retriever(search_type="mmr")

def slow_echo(message, history):
    for i in range(len(message)):
        yield "You typed: " + message[: i+1]


demo = gr.ChatInterface(slow_echo).queue()

if __name__ == "__main__":
    demo.launch()

ModuleNotFoundError: No module named 'gradio'